## Working with Snapshot camera

Idea: find only the large cluster to reduce detection noise. Then, use the lifespan of customer's cluster to find the most probable vehicle in the snapshot.
Snapshot cam: takes photo every 1:15 second

In [1]:
import sys

sys.path.append("/home/jeans/internship/parking-customer-count")

from countpassenger.Config import conf
from countpassenger import Preprocess
from countpassenger import Cluster

import countpassenger
import pandas as pd
import numpy as np

import os.path as osp
import os

pd.set_option("display.precision", 10)
CURRENT_CAM = "mbk-14-11"

df_vehicle = pd.read_csv(
    osp.join(conf.RESOURCES_RAW_DIR, "2024-04-28/mbk-tourist-vehicle-object-20240428.csv")
)
df_cross = pd.read_csv(
    osp.join(conf.RESOURCES_RAW_DIR, "2024-04-28/mbk-tourist-raw-cross-object-20240428.csv")
)
df_reverse = pd.read_csv(
    osp.join(conf.RESOURCES_RAW_DIR, "2024-04-28/mbk-tourist-raw-reverse-object-20240428.csv")
)

df_vehicle = Preprocess.df_clean_vehicle(
    df_vehicle=df_vehicle,
)
df_vehicle = Preprocess.filter_camera(df_vehicle, camera_name=CURRENT_CAM + "-vehicle-snapshot")
df_vehicle = Preprocess.sort_df(df_vehicle, conf.VEHICLE_INTEREST_NON_SNAPSHOT)

# print(df_vehicle[[*conf.VEHICLE_INTEREST_SNAPSHOT]].to_string())

In [2]:
df_cross = Preprocess.df_clean_customer(df_cross)
df_cross = Preprocess.filter_camera(df_cross, camera_name=CURRENT_CAM)
df_cross = Preprocess.sort_df(df_cross, conf.CUSTOMER_INTEREST_SNAPSHOT)

# print(df_cross[[*conf.CUSTOMER_INTEREST_SNAPSHOT,'timestamp_unix']].to_string())

df_reverse = Preprocess.df_clean_customer(df_reverse)
df_reverse = Preprocess.filter_camera(df_reverse, camera_name=CURRENT_CAM)
df_reverse = Preprocess.sort_df(df_reverse, conf.CUSTOMER_INTEREST_SNAPSHOT)

# print(df_reverse[[*conf.CUSTOMER_INTEREST_SNAPSHOT, "timestamp_unix"]].to_string())

### Cluster only large vehicle 

In [3]:
params = dict(
    min_cluster_size=1000, #have at least 5 people together
    min_samples=None,
    cluster_selection_epsilon=0,  # math.sqrt(1440**2 + 2560**2) + _TIME_BIAS,
    max_cluster_size=40,
    # specify yourown
    # metric="euclidean",
    # metric_params=None,
    alpha=1.0,
    algorithm="brute",
    leaf_size=40,
    n_jobs=None,
    cluster_selection_method="eom",
    allow_single_cluster=False,
    store_centers="medoid",
    copy=False,
)

df_cross, cluster_cross = Cluster.perform_cross_clustering(df_cross=df_cross,params=params)
print(Preprocess.sort_df(cluster_cross, conf.CLUSTER_INTEREST)[conf.CLUSTER_INTEREST].to_string())

/home/jeans/internship/parking-customer-count/countpassenger/Cluster.py:60: RuntimeWarning: overflow encountered in exp
  exponential_part = np.exp(x - n) + n - 1
/home/jeans/internship/parking-customer-count/countpassenger/Cluster.py:58: RuntimeWarning: overflow encountered in exp
  sigmoid = 1 / (1 + np.exp(-k * (x - n)))


           timestamp_min        timestamp_max  count  cluster_id        xmid_mean       ymid_mean
1    2024-04-28 02:00:18  2024-04-28 02:35:01      4           0  1330.6250000000  651.8750000000
5    2024-04-28 02:56:10  2024-04-28 02:57:19      2           4  1802.0000000000  430.5000000000
89   2024-04-28 03:02:30  2024-04-28 03:02:37      2          88  1653.5000000000  483.0000000000
90   2024-04-28 03:02:48  2024-04-28 03:03:17      8          89  1645.1875000000  501.9375000000
60   2024-04-28 03:03:37  2024-04-28 03:03:39      2          59  1750.0000000000  522.0000000000
0    2024-04-28 03:04:07  2024-04-28 16:48:21     38          -1  1661.7105263158  449.1052631579
25   2024-04-28 03:04:57  2024-04-28 03:05:37      2          24  1609.2500000000  484.2500000000
17   2024-04-28 03:07:01  2024-04-28 03:07:04      2          16  2100.7500000000  213.2500000000
18   2024-04-28 03:10:34  2024-04-28 03:11:39      3          17  1538.3333333333  567.8333333333
19   2024-04-28 03:1

/home/jeans/anaconda3/envs/ml_env/lib/python3.12/site-packages/sklearn/cluster/_hdbscan/hdbscan.py:264: UserWarning: The minimum spanning tree contains edge weights with value infinity. Potentially, you are missing too many distances in the initial distance matrix for the given neighborhood size.
  warn(
/home/jeans/internship/parking-customer-count/countpassenger/Cluster.py:60: RuntimeWarning: overflow encountered in exp
  exponential_part = np.exp(x - n) + n - 1
/home/jeans/internship/parking-customer-count/countpassenger/Cluster.py:58: RuntimeWarning: overflow encountered in exp
  sigmoid = 1 / (1 + np.exp(-k * (x - n)))
/home/jeans/anaconda3/envs/ml_env/lib/python3.12/site-packages/sklearn/cluster/_hdbscan/hdbscan.py:963: RuntimeWarning: invalid value encountered in multiply
  dist_mat = dist_mat * strength
/home/jeans/internship/parking-customer-count/countpassenger/Cluster.py:58: RuntimeWarning: overflow encountered in exp
  sigmoid = 1 / (1 + np.exp(-k * (x - n)))
/home/jeans/in